# FDC Estimation

This notebook contains the main code for computing FDCs by the different methods. 

It requires the runoff statistics to have been computed, and the results of the XGBoost prediction model (catchment attributes $\rightarrow$ hydrologic signatures / runoff statistics) to have been processed in Notebook 3.  It also requires the pre-processing of reference (baseline) distributions by KDE for validation from Notebook 4.

In [1]:
import os
import pandas as pd
import numpy as np
import json
from time import time
import geopandas as gpd

import xgboost as xgb
xgb.config_context(verbosity=2)

from collections import defaultdict
from multiprocessing import Pool, cpu_count

import jax
import jax.numpy as jnp

from utils.kde_estimator import KDEEstimator
from utils.knn_estimator import kNNEstimator
from utils.LSTM_estimator import LSTMFDCEstimator
from utils.parametric_estimator import ParametricFDCEstimator
from utils.fdc_estimator_context import FDCEstimationContext 
from utils.fdc_data import StationData
from utils.evaluation_metrics import EvaluationMetrics

import utils.data_processing_functions as dpf

from pathlib import Path
BASE_DIR = os.getcwd()

In [2]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot

import xyzservices.providers as xyz
tiles = xyz['USGS']['USTopo']
output_notebook()


Loading BokehJS ...

In [3]:
import warnings
import sys
import traceback

def warn_with_traceback(message, category, filename, lineno, file=None, line=None):
    log = file if hasattr(file, 'write') else sys.stderr
    traceback.print_stack(file=log)
    log.write(warnings.formatwarning(message, category, filename, lineno, line))

warnings.showwarning = warn_with_traceback

In [4]:
# load the catchment characteristics
fname = f'catchment_attributes_with_runoff_stats.csv'
attr_df = pd.read_csv(os.path.join('data', fname), dtype={'official_id': str, 'drainage_area_km2': float})
attr_df.columns = [c.lower() for c in attr_df.columns]
attr_df['log_drainage_area_km2'] = np.log(attr_df['drainage_area_km2'])
# attr_df = attr_df[~attr_df['official_id'].isin(exclude)]
# attr_df.columns = [c.lower() for c in attr_df.columns]
attr_df['tmean'] = (attr_df['tmin'] + attr_df['tmax']) / 2.0
station_ids = attr_df['official_id'].values
stn_da_dict = attr_df.set_index('official_id')['drainage_area_km2'].to_dict()
# assert '12414900' in station_ids

print(f'There are {len(station_ids)} monitored basins in the attribute set.')

There are 1098 monitored basins in the attribute set.


In [5]:
# streamflow folder from (updated) HYSETS
HYSETS_DIR = Path('/home/danbot/code/common_data/HYSETS')
# STREAMFLOW_DIR = HYSETS_DIR / 'streamflow'

hs_df = pd.read_csv('data/HYSETS_watershed_properties.txt', sep=';', dtype={'Official_ID': str})
hs_df = hs_df[hs_df['Official_ID'].isin(station_ids)]
hs_df.head(2)

,Watershed_ID,Source,Name,Official_ID,Centroid_Lat_deg_N,Centroid_Lon_deg_E,Drainage_Area_km2,Drainage_Area_GSIM_km2,Flag_GSIM_boundaries,Flag_Artificial_Boundaries,...,Land_Use_Wetland_frac,Land_Use_Water_frac,Land_Use_Urban_frac,Land_Use_Shrubs_frac,Land_Use_Crops_frac,Land_Use_Snow_Ice_frac,Flag_Land_Use_Extraction,Permeability_logk_m2,Porosity_frac,Flag_Subsoil_Extraction
846,847,HYDAT,CROWSNEST RIVER AT FRANK,05AA008,49.59732,-114.4106,402.6522,NaN,0,0,...,0.0103,0.0065,0.0328,0.0785,0.0015,0.0002,1,-15.543306,0.170479,1
849,850,HYDAT,CASTLE RIVER NEAR BEAVER MINES,05AA022,49.48866,-114.1444,820.6510,NaN,0,0,...,0.0058,0.0023,0.0105,0.1156,0.0246,0.0000,1,-15.929747,0.150196,1


In [6]:
# load the baseline PMFs from the previous notebook
pmf_path = Path(os.getcwd()) / 'data' / 'results' / 'baseline_distributions' / f'bcub_pmfs.csv'
pmf_df = pd.read_csv(pmf_path, index_col=0)
pmf_stations = pmf_df.columns
station_ids = list(set(station_ids).intersection(set(pmf_stations)))
print(len(station_ids))

1097


In [7]:
# Exclude catchments found to be regulated (missed by QC)
# 12143700 isnot actually a dam, it's just basically a seepage monitoring station
# from a small catchment next to a dam 
dam_sites = ['12398000', '12058800', '12143700', '12323760'] 
official_ids_to_include = [s for s in pmf_stations if s not in dam_sites]

In [8]:
# retrieve LSTM ensemble predictions
LSTM_ensemble_result_folder = '/home/danbot/code/neuralhydrology/data/ensemble_results_20250514' # based on mean NSE loss
# LSTM_ensemble_result_folder = '/home/danbot/code/neuralhydrology/data/ensemble_results_20250627' # based on 95% NSE loss
lstm_result_files = os.listdir(LSTM_ensemble_result_folder)
lstm_result_stns = [e.split('_')[0] for e in lstm_result_files]
assert '12414900' in lstm_result_stns

# find any non-matching station ids in the lstm result files
for stn in lstm_result_stns:
    if stn not in station_ids:
        # try adding a leading zero
        ending_in = [e for e in station_ids if e.endswith(stn)]
        if len(ending_in) > 0:
            print(stn, 'matches', ending_in)
        modified_stn = stn.zfill(8)
        if modified_stn in station_ids:
            print(f'Found modified station id: {modified_stn} for {stn}')
        else:
            print(f'Warning: {stn} is in LSTM results but not in the station attributes.')

# filter for the common stations between BCUB region and LSTM-compatible (i.e. 1980-)
daymet_concurrent_stations = list(set(station_ids) & set(lstm_result_stns) & set(official_ids_to_include))
# assert '12414900' in daymet_concurrent_stations
print(f'There are {len(daymet_concurrent_stations)} monitored basins concurrent with LSTM ensemble results.')
print(f'There are {len(pmf_stations)} monitored basins with baseline PMFs.')

There are 719 monitored basins concurrent with LSTM ensemble results.
There are 1097 monitored basins with baseline PMFs.


In [9]:
for stn in pmf_stations:
    hs_data = hs_df[hs_df['Official_ID'] == stn].copy()
    name = hs_data['Name']
    if 'DAM' in name:
        print(stn, name)
    if 'RESERVOIR' in name:
        print(stn, name)


In [10]:
# import updated catchment polygons
# poly_fpath = os.path.join(os.path.join('data', f'BCUB_watershed_attributes_updated_{rev_date}.csv'))
# catchment_gdf = pd.read_csv(poly_fpath)
# catchment
# catchment_gdf = catchment_gdf[catchment_gdf['Official_ID'].isin(station_ids)]
# print(len(catchment_gdf), 'catchments in the polygon set')

# import the license water extraction points
# dam_gdf = gpd.read_file('data/Dam_Points_20240103.gpkg')
# assert dam_gdf.crs == catchment_gdf.crs, "Catchment and dam geometries must have the same CRS"
# joined = gpd.sjoin(catchment_gdf, dam_gdf, how="inner", predicate="contains")
# Create a new boolean column 'contains_dam' in catchment_gdf.
# If a polygon's index appears in the joined result, it means it contains at least one point.
# regulated = joined['Official_ID'].values
# catchment_gdf["contains_dam"] = catchment_gdf['Official_ID'].apply(lambda x: x in regulated)
# n_regulated = catchment_gdf['contains_dam'].sum()
# print(f'{n_regulated}/{len(catchment_gdf)} catchments contain withdrawal licenses')

# # create dicts for easier access to 'official_id': 'drainage area', geometry, regulation status
# da_dict = attr_df[['official_id', 'drainage_area_km2']].copy().set_index('official_id').to_dict()['drainage_area_km2']
# dam_dict = catchment_gdf[['Official_ID', 'contains_dam']].copy().set_index('Official_ID').to_dict()['contains_dam']
# polygon_dict = catchment_gdf[['Official_ID', 'geometry']].copy().set_index('Official_ID').to_dict()['geometry']

# # add the centroid point geometry to the attributes dataframe
# attr_df = attr_df[attr_df['official_id'].isin(catchment_gdf['Official_ID'].values)].copy()
# centroids = attr_df.apply(lambda x: polygon_dict[x['official_id']].centroid, axis=1)
# attr_gdf = gpd.GeoDataFrame(attr_df, geometry=centroids, crs=catchment_gdf.crs)
# attr_gdf["contains_dam"] = attr_gdf['official_id'].apply(lambda x: dam_dict[x] if x in dam_dict else False)
# add the concurrency status as a boolean column
# attr_df['LSTM_concurrent'] = attr_df['official_id'].apply(lambda x: x in daymet_concurrent_stations)
# attr_df.reset_index(inplace=True, drop=True)
# print(f'N network stations={len(attr_df)}')

## Non-Parametric Simulation

### Time-based ensemble

A probability distribution $\hat p = f(\tilde x(t))$ is estimated for a target (ungauged location) by a weighted mean of runoff time-series from k nearest neighbour stations, $\tilde x(t) = \textbf{X}(t)\cdot w$ where $X(t) \in \mathbb{R}^{N \times k}$ and $w \in \mathbb{R}^{k\times 1}$ is a vector of k weights.  So $\hat p = f(\textbf{X}(t) \cdot w )$  Weights $w$ are computed in three ways, described in the next subsection, and k-nearest neighbours are selected using the criteria defined below.  Each gauged station in the monitoring network is treated as an ungauged location to generate a large sample of simulations across hydrologically diverse catchments, or rather as many catchments as can be tested.

### Frequency-based ensembles

A simulated probability density function is estimated from observations of k nearest neighbour stations.  First, k simulated series are generated by equal unit area runoff , $\hat p = \hat P \cdot w$ where $\hat P = [\hat p_1, \hat p_2, \cdots, \hat p_k]$ and each $\hat p_i = f(X_i(t))$.

In both cases, the function $f \rightarrow \hat p(x)$ represents kernel density estimation, which defines the probability density as $$\hat p(x) = \frac{1}{n \cdot h(x)} \sum_{i=1}^{n}K\left( \frac{x-x_i}{h(x)}\right)$$ 

Where $h(x)$ reflects an adaptive kernel bandwidth that addresses vestiges of precision in the observed data to reflect the nature of streamflow as a continuous variable, and additionally incorporates piecewise linear model to represent overall measurement uncertainty.


## Notes on k-nearest neighbours

Time series streamflow records vary widely in their temporal coverage, and finding k-nearest neighbours presents a tradeoff between selecting nearest neighbours and maximizing the number of observations concurrent with the target.  From the literature, concurrency is assured by pre-selecting a subset of stations with continuous records over a common period of record, or by infilling gaps with k-nearest neighbours simulation.  Some kind of tradeoff must be made, and we aim to use a method that maximizes information content while minimizing the number of assumptions.  The following notes are intended to clarify the implications of using k-nearest neighbours to fill gaps in the time series.

1. **Infilled-by-kNN != Independent Proxy**: If a gap in an observation record is inferred from neighbors, it becomes redundant in the ensemble and increases the weight of the other (k minus n) neighbours.  So at that time step, its influence is non-unique, and including it in the ensemble is functionally equivalent to using the same set of other proxies directly, or just reducing the ensemble size.

2. **Inflated Ensemble Size**: Filling gaps by "nested" k-nearest neighbours inflates the expresed number of independent neighbors.  Comparing the effectiveness of ensemble simulations as a function of k is then misleading because the effective number of independent proxies is *at most* k. 

3. **Information leakage risk**: If you repeatedly use kNN to fill missing data from within the same pool, especially when simulating extreme values, you risk suppressing variability by biasing toward the central tendency of the ensemble.  This defeats one of the core motivations for kNN: to preserve structure and variability from observations at neighboring stations.

To address the nuance above, we propose three time-based methods for selecting k-nearest neighbours beyond strictly nodes in the network.  The problem is related to the set-cover problem where the goal is to select a subset of stations that maximizes the intersection of their data availability over a specified time period.  The following sections outline the three methods for selecting k-nearest neighbours based on availability of concurrent data.

### Summary: Set-Theoretic Foundations of Strict k-NN Concurrency Selection

This problem is closely related to classic combinatorial and set-theoretic optimization problems.

#### Set-Theoretic Definition

Let each column $( S_i \subseteq T )$ represent the set of timestamps where station $( i )$ has valid (non-NaN) data.  
Let $( \mathcal{S} = \{ S_1, S_2, \dots, S_n \} )$ be the collection of all such subsets, sorted by proximity (e.g., distance or attribute similarity).  
The goal is to select a subset $( \mathcal{K} \subset \mathcal{S} )$ such that:
- $( |\mathcal{K}| = k )$
- $( \bigcap_{S \in \mathcal{K}} S )$ satisfies a temporal completeness constraint (e.g., ≥5 years with ≥10 observations in each of 12 months)

This is a constrained subset selection problem on the intersection of sets.

#### Related Concepts

| Concept                                 | Description |
|----------------------------------------|-------------|
| Set Intersection Selection             | Select \( k \) sets whose intersection satisfies a completeness constraint. |
| Maximum Coverage under Cardinality Constraint | Choose \( k \) sets to maximize the coverage (or completeness) of their intersection. |
| Recursive k-Subset Validation          | If the initial \( k \) sets fail, iteratively add more candidates and evaluate all \( \binom{k+1}{k} \) combinations, and so on. |
| NP-Hard Nature                         | This problem is computationally hard and shares structure with the Set Cover and Maximum Coverage problems. |

#### Practical Implication

This formulation justifies using greedy or approximate subset selection strategies when exhaustively testing all combinations becomes computationally infeasible.
## Define a universal parametric prior

In order to fairly test how parametric and non-parametric pdf estimation methods compare to each other, we need a consistent way to deal with indeterminate cases where the simulated distribution does not provide support coverage of the "ground truth" observations.  I feel two ways about this: the KL divergence is the culprit here, and the problem could be avoided by choosing another divergence measure.  However the definintion of KL divergence in information theoretic terms of compression make it seem more foundational than other measures, but ultimately is this true?  Should we look to math statistics to make more direct links between f-divergences and what we use as a discriminant for a particular application?  Should we be more concerned about "Bayesian consistency" of the discriminant (or surrogate loss function) with the choice of divergence measure?


1.  **Quantify the distribution of unsupported mass across all models**.  It is important to describe the extent of the problem across the sample **and** across various methods.  i.e. discrete distributions have the issue of support coverage, but so do all methods!
2.  Even in kNN / ensemble simulation approaches, the problem of incomplete support coverage necessitates assuming some prior probability.  The issue is that setting a uniform prior over the observed range takes advantage of information about the observed range.




### Global Uniform Prior

$$f(x) = \frac{1}{b-a}, \quad x\in (a, b) \text{ and } f(x) = 0 \text{ otherwise.}$$
$$\int_a^b f(x)\text{dx} = 1$$

Given the target range is a sub interval $(c, d) \subseteq (a, b)$, then the **total** prior probability mass over (c, d) is:

$$M_\text{target} = \int_c^d \frac{1}{b-a}\text{dx} = \frac{d-c}{b-a}$$

Over the set of intervals $\Delta x_i$ covering the **target range**, the probability mass associated with each interval (bin) is given by: 

$$\Delta x_i \frac{d-c}{b-a}$$



A desirable property of the prior is that it reflects the strength of belief in the model (data), where a smaller prior reflects stronger belief in the data/model and vice versa.  Dividing by the number of observations has such an effect, however it also makes for very small priors.  The consequence of very small priors is they have negligible effect on models that provide complete support coverage, and they severely penalize models that do not, resulting in a form of instability.  The very small prior creates a heavy tail in the distribution of a large sample of KL divergences, with further downstream effects in optimization.  

A method that uses a prior with negligible effect on a model with complete support coverage and a very big effect on one without can be interpreted in a few ways:  

1.  Incomplete support coverage, or underspecification, is very heavily penalized.  The method does not tolerate a model that cannot predict the full observed range.
2.  A **proper** probability distribution sums (discrete) or integrates (continuous) to 1.  Very small probabilities are in a sense associated with a high degree of certainty since they reflect the expectation of the system being observed in a particular state.
3.  The penalty of underestimating a state frequency is that storing and transmitting information about the state requires (the log ratio) more bandwidth/disk space because it is assigned a longer bit string than the actual frequency calls for under optimal encoding.
4.  Assigning a very small probability to a state ...

In [11]:
# load the predicted parameter results
parameter_prediction_results_folder = os.path.join('data', 'results', 'parameter_prediction_results', )
predicted_params_fpath   = os.path.join(parameter_prediction_results_folder, 'mean_parameter_predictions.csv')
rdf = pd.read_csv(predicted_params_fpath, index_col=['official_id'], dtype={'official_id': str})
predicted_param_dict = rdf.to_dict(orient='index')
predicted_param_dict['0212414900'].keys()

dict_keys(['uar_mean_mean_predicted', 'uar_mean_actual', 'uar_std_mean_predicted', 'uar_std_actual', 'uar_median_mean_predicted', 'uar_median_actual', 'uar_mad_mean_predicted', 'uar_mad_actual', 'log_uar_mean_mean_predicted', 'log_uar_mean_actual', 'log_uar_std_mean_predicted', 'log_uar_std_actual', 'log_uar_median_mean_predicted', 'log_uar_median_actual', 'log_uar_mad_mean_predicted', 'log_uar_mad_actual'])

In [12]:
plots = []
predicted_param_sample = {}
for l, al in zip(['log_uar_mean_mean_predicted', 'log_uar_std_mean_predicted'], [r'$$\text{Log Mean UAR }(L/s/\text{km}^2)$$', r'$$\text{Log SD UAR }(L/s/\text{km}^2)$$']):
    vals = [d[l] for _, d in predicted_param_dict.items()]
    predicted_param_sample[l] = vals
    # plot the histogram of the mean_uar values
    hist, edges = np.histogram(vals, bins=40, density=True)
    # create a scatter plot of the predicted parameter vs the target parameter
    f = figure(title=f'Predicted {l}', width=600, height=400)
    f.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], fill_color='lightblue', line_color='black', legend_label='')
    f.xaxis.axis_label = al
    f.yaxis.axis_label = r'$$P(x)$$'
    f = dpf.format_fig_fonts(f, font_size=14)
    plots.append(f)
# retrieve all the mean_uar values 

lt = gridplot(plots, ncols=2, width=400, height=400)
show(lt)

In [13]:
class FDCEstimatorRunner:
    def __init__(self, stn_id, ctx, methods, k_nearest, parametric_target_cols, estimator_classes, **kwargs):
        self.stn_id = stn_id
        self.ctx = ctx
        self.methods = methods
        self.k_nearest = k_nearest
        self.parametric_target_cols = parametric_target_cols
        self._create_results_folders()
        self._create_readme()
        self.ESTIMATOR_CLASSES = estimator_classes
        self.prior_strength = ctx.prior_strength

    def _create_results_folders(self):
        # create a results foder for each method if it doesn't exist
        self.results_folder = os.path.join('data', 'results', f'fdc_estimation_results',)
        for method in self.methods:
            method_folder = os.path.join(self.results_folder, method)
            if method == 'lstm':
                self.lstm_rev_date = self.ctx.LSTM_ensemble_result_folder.split('_')[-1]
                method_folder = os.path.join(self.results_folder, f'{method}_{self.lstm_rev_date}')
            if not os.path.exists(method_folder):
                os.makedirs(method_folder)

    
    def _create_readme(self):
        # create a readme file in the results folder to list constraints
        readme_file = os.path.join(self.results_folder, 'README.txt')
        
        with open(readme_file, 'w') as file:
            file.write("This folder contains the results of the FDC estimation.\n")
            file.write(f"Methods evaluated: {', '.join(self.methods)}\n")
            # add the concurrency constraint and number of stations represented in the network
            N = len(self.ctx.official_ids)
            if self.ctx.include_pre_1980_data == False:
                file.write(f'Uses only stations within Daymet input period of record / LSTM results: N={N} stations in the network.\n')
                file.write(f'Global start date on streamflow data: {self.ctx.global_start_date}\n')
            else:
                file.write(f'Uses all available network stations in the BCUB region (1950-2024): N={N} stationsin the network.')
                

    def _load_reference_distributions(self):
        self.kde = KDEEstimator(self.data.baseline_log_grid, self.data.log_dx)
        self.baseline_pmf, self.baseline_pdf = self.data.baseline_pmf, self.data.baseline_pdf        
        self.ctx.baseline_pmf = self.baseline_pmf


    def _save_result(self, result):
        with open(self.result_file, 'w') as file:
            json.dump(result, file, indent=4)

 
    def run_selected(self):
        # check the minimum number of years of overlap for all stations in self.ctx.overlap_dict
        for method in self.methods:
            self.result_file = os.path.join(self.results_folder, method, f'{self.stn_id}_fdc_results.json')
            if method == 'lstm':
                method_folder = f'{method}_{self.lstm_rev_date}'
                self.result_file = os.path.join(self.results_folder, method_folder, f'{self.stn_id}_fdc_results.json')
            
            if os.path.exists(self.result_file):
                continue
            else:
                self.data = StationData(self.ctx, self.stn_id)
                self.data.k_nearest = self.k_nearest
                self.data.parametric_target_cols = self.parametric_target_cols
                self._load_reference_distributions()

                eval_metrics = EvaluationMetrics(self.data.baseline_log_grid, self.data.log_dx)
                EstimatorClass = self.ESTIMATOR_CLASSES[method]
                estimator = EstimatorClass(
                    self.ctx, self.stn_id, self.data
                )
                self.data.eval_metrics = eval_metrics
                result = estimator.run_estimators()
                self._save_result(result)
            # except Exception as e:
            #     raise Exception(f"  {method} estimator failed for {self.stn_id}: {str(e)}")
                

In [20]:
np.random.seed(42)

target_cols = [
    'mean_uar', 'sd_uar', 
    'mean_logx', 'sd_logx', 
]

# from utils import FDCEstimationContext
attr_df_fpath = os.path.join('data', f'catchment_attributes_with_runoff_stats.csv')
LSTM_forcings_folder = '/home/danbot/neuralhydrology/data/BCUB_catchment_mean_met_forcings_20250320'
baseline_distribution_folder = os.path.join('data', 'results', 'baseline_distributions')
# parameter_prediction_results_folder = os.path.join('data', 'parameter_prediction_results')

methods = ('parametric', 'lstm',)# 'knn',)
# methods = ('knn',)
k_nearest = 10
include_pre_1980_data = True  # use only stations with data 1980-present concurrent with Daymet
daymet_start_date = '1980-01-01'  # default start date for Daymet data
if include_pre_1980_data:
    daymet_start_date = '1950-01-01'

processed = []
ESTIMATOR_CLASSES = {
    'parametric': ParametricFDCEstimator,
    'lstm': LSTMFDCEstimator,
    'knn': kNNEstimator,
    # add others here
}
input_data = {
    'attr_df_fpath': attr_df_fpath,
    'LSTM_forcings_folder': LSTM_forcings_folder,
    'LSTM_ensemble_result_folder': LSTM_ensemble_result_folder,
    'include_pre_1980_data': include_pre_1980_data,  # use only stations with data 1980-present concurrent with Daymet
    'predicted_param_dict': predicted_param_dict,
    'divergence_measures': ['DKL', 'EMD'],
    'eps': 1e-12,
    'min_flow': 1e-4,
    'n_grid_points': 2**12,
    'min_record_length': 5,
    'minimum_days_per_month': 20,
    'parametric_target_cols': target_cols,
    'all_station_ids': official_ids_to_include,
    'daymet_concurrent_stations': daymet_concurrent_stations,
    'baseline_distribution_folder': baseline_distribution_folder,
    'prior_strength': 1e-2,  # prior strength for the Laplace fit
}

context = FDCEstimationContext(**input_data)

    Using all stations in the catchment data with a baseline PMF (validated): 1093
    ...overlap dict loaded from data/record_overlap_dict.json


In [21]:
processed = []
# missing_lstm_set1 =['12414900']
# missing_lstm_set2 = ['10AA002']#, '05BG002', '08OA005', '12172000', '12173500', '12422000']

t0 = time()
process_fdcs = True
if process_fdcs:
    print('Processing FDCs...')
    for stn in daymet_concurrent_stations:
        # if stn in missing_lstm_set2: # this station has no data in the LSTM ensemble results
        #     print(f'    ...skipping {stn} due to naming issue.')
        #     continue
        print(f'Estimating FDC for {stn}...')
        foo = predicted_param_dict[stn]['uar_mean_actual']
        print(f'   {stn}: actual mean UAR={foo:.2f} L/s/km2')
        runner = FDCEstimatorRunner(stn, context, methods, k_nearest, target_cols, ESTIMATOR_CLASSES)
        runner.run_selected()
        processed.append(stn)
        if len(processed) % 10 == 0:
            t1 = time()
            elapsed = t1 - t0
            unit_time = elapsed / len(processed)
            print(f'Processed {len(processed)}/{len(context.official_ids)} stations in {unit_time:.2f} seconds per station')

Processing FDCs...
Estimating FDC for 08GE002...
   08GE002: actual mean UAR=52.19 L/s/km2
Estimating FDC for 10AA005...
   10AA005: actual mean UAR=7.65 L/s/km2
Estimating FDC for 12125200...
   12125200: actual mean UAR=20.99 L/s/km2
Estimating FDC for 08ME028...
   08ME028: actual mean UAR=52.02 L/s/km2
Estimating FDC for 12087000...
   12087000: actual mean UAR=31.58 L/s/km2
Estimating FDC for 10BD001...
   10BD001: actual mean UAR=8.73 L/s/km2
Estimating FDC for 07OC001...
   07OC001: actual mean UAR=2.75 L/s/km2
Estimating FDC for 08NC004...
   08NC004: actual mean UAR=48.61 L/s/km2
Estimating FDC for 05BF019...
   05BF019: actual mean UAR=10.93 L/s/km2
Estimating FDC for 08FB006...
   08FB006: actual mean UAR=11.71 L/s/km2
Processed 10/1093 stations in 6.60 seconds per station
Estimating FDC for 12389500...
   12389500: actual mean UAR=7.38 L/s/km2
Estimating FDC for 12102075...
   12102075: actual mean UAR=49.70 L/s/km2
Estimating FDC for 07EE007...
   07EE007: actual mean UAR=

## Example case of bad KDE fit due to sparse support and precision vestiges



In [22]:

def generate_pdf_fig(y1, test_data, ts):

    # check y1 (pdf) integrates to 1 over baseline_log_grid
    y1_integral = np.trapezoid(y1, x=test_data.baseline_log_grid)

    # set up a histogram of the observed data
    xx = np.linspace(np.min(np.log(ts)), np.max(np.log(ts)), 50)
    hist, edges = np.histogram(np.log(ts), bins=xx, density=True)
    # compute the maximum bin probability mass
    dx = edges[1] - edges[0]
    max_bin_prob = hist.max() * dx  # bin width
    foo = hist * dx
    print(f'Max bin probability mass: {max_bin_prob:.4f}, y1 integral: {y1_integral:.4f}')

    # plot the adaptive PMF and the FFTKDE result
    edges = np.exp(edges)
    # f = figure(title=f'{target_id}', width=600, height=400, x_axis_type='log')
    f = figure(title=f'', width=600, height=400, x_axis_type='log')
    f.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], 
        fill_color='lightblue', line_color='dodgerblue', 
        legend_label='Observed Histogram')
    # x_lin = np.exp(test_data.baseline_log_gr)
    f.line(test_data.baseline_lin_grid, adaptive_pdf, line_width=2, color='green', legend_label='Adaptive BW')
    f.line(test_data.baseline_lin_grid, y1, line_width=3, color='black', line_dash='dashed', legend_label='FFT KDE')
    f.xaxis.axis_label = 'UAR (L/s/km2)'
    f.yaxis.axis_label = 'Density'
    f.legend.location = 'top_left'
    f.legend.background_fill_alpha = 0.25
    f.legend.click_policy = 'hide'
    f = dpf.format_fig_fonts(f, font_size=14)
    return f

In [23]:
from KDEpy import FFTKDE
bad_klds = ['05AB022', '12345000', '12353820', '08NL037', '08NA056', '08NA052', '12111500',
            '08NA053', '08NH100', '12367500']
bad_rmses = ['15031000', '12345000', '12067500', '08HB045', '15058700', '12353820',
           '15052000', '15195000', '15067900', '08HA069']
bad_nses = ['12353820', '12345000', '12091060', '12067500', '12111500', '08HB045',
            '08HB047', '15031000', '10CD004', '12301550']
# plots = []
# for target_id in bad_nses:
#     target_da= stn_da_dict[target_id]
#     test_data = StationData(context, target_id)
#     ts = test_data.stn_df[f'{target_id}_uar'].dropna().values

#     # get the unique UAR values
#     unique_vals = sorted(np.unique(ts))

#     df =  test_data.stn_df.copy()
#     df = df[df['zero_flow_flag'] == True].copy()


#     kde_obj = KDEEstimator(test_data.baseline_log_grid, test_data.log_dx)
#     adaptive_pmf, adaptive_pdf = kde_obj.compute(ts, target_da)


#     y1 = FFTKDE(bw="silverman").fit(np.log(ts)).evaluate(test_data.baseline_log_grid)
#     f = generate_pdf_fig(y1, test_data, ts)
#     plots.append(f)


In [ ]:
lt = gridplot(plots, ncols=2, width=550, height=400)
show(lt)


### Compute the complexity of searching the space of k-nearest neighbours for the optimal ensemble


In [ ]:
from math import comb
# from bokeh.plotting import figure, show, output_file
from bokeh.models import ColorBar, BasicTicker, HoverTool, ColumnDataSource
from bokeh.transform import linear_cmap
from bokeh.palettes import Viridis256

def plot_search_space_heatmap(N_min=10, N_max=1000, N_step=10, k_max=10):# output_path="search_space_heatmap.html"):
    """
    Generate and save a Bokeh heatmap of log10(search space size) for ensembles of size 1 to k_max
    from a network of size N, excluding the target node.

    Parameters:
    - N_min (int): Minimum network size.
    - N_max (int): Maximum network size.
    - N_step (int): Step between network sizes.
    - k_max (int): Maximum ensemble size to consider.
    - output_path (str): Path to save the HTML output.
    """
    N_values = np.arange(N_min, N_max + 1, N_step)
    k_values = np.arange(1, k_max + 1)

    data = []
    for N in N_values:
        for k in k_values:
            max_j = min(k, N - 1)
            size = sum(comb(N - 1, j) for j in range(1, max_j + 1))
            data.append((N, k, size))

    df = pd.DataFrame(data, columns=['N', 'k', 'search_space'])
    df['log_search_space'] = np.log10(df['search_space'].astype(float).replace(0, np.nan))

    source = ColumnDataSource(df)

    mapper = linear_cmap(field_name='log_search_space', palette=Viridis256,
                         low=df['log_search_space'].min(), high=df['log_search_space'].max())

    hover = HoverTool(
        tooltips=[
            ("N", "@N"),
            ("k", "@k"),
            ("Search space", "@search_space{0,0}"),
            ("log10(Search space)", "@log_search_space{0.00}")
        ]
    )

    p = figure(title="Log10 of Search Space Size vs N and k",
               x_axis_label='Network size N',
               y_axis_label='Ensemble size k',
               x_range=(N_min, N_max), y_range=(1, k_max),
               width=900, height=550,
               tools=['pan', 'wheel_zoom', 'box_zoom', 'reset', hover])

    p.rect(x="N", y="k", width=N_step, height=1, source=source,
           line_color=None, fill_color=mapper)

    color_bar = ColorBar(color_mapper=mapper['transform'], ticker=BasicTicker(),
                         label_standoff=12, location=(0, 0), title='log10(search space)')
    p.add_layout(color_bar, 'right')
    return p

In [ ]:
# fig = plot_search_space_heatmap()
# show(fig)

### Evaluate how biased a quantile-based spread estimate is at different resolutions

In [ ]:

def percentile_spread_convergence(obs, percentiles_list=None):
    """
    Compare the standard deviation of observed data to that estimated from percentiles.

    Parameters:
    - obs: 1D array-like of observed values
    - percentiles_list: list of lists/arrays of percentiles to compute (0–100).
      If None, defaults to increasingly fine resolutions.

    Returns:
    - pd.DataFrame with columns:
        'num_percentiles', 'percentiles', 'spread_std', 'sample_std', 'relative_error'
    """
    obs = np.asarray(obs)
    sample_std = np.std(obs, ddof=0)

    if percentiles_list is None:
        percentiles_list = [np.arange(10, 100, 10),
                            np.arange(5, 100, 5),
                            np.arange(2, 100, 2),
                            np.arange(1, 100, 1),
                           np.arange(0.1, 99.9, 0.1),
                           np.arange(0.01, 99.99, 0.01), 
                           np.linspace(0, 100, 1000)]

    results = []
    for pct_set in percentiles_list:
        q_vals = np.percentile(obs, pct_set)
        spread_std = np.std(q_vals)
        rel_error = (spread_std - sample_std) / sample_std
        results.append({
            'num_percentiles': len(pct_set),
            'percentiles': pct_set,
            'spread_std': spread_std,
            'sample_std': sample_std,
            'relative_error': rel_error
        })

    return pd.DataFrame(results)

In [ ]:
target_id = '08MH147'
target_da= stn_da_dict[target_id]
test_data = StationData(context, target_id)

# get the unique UAR values
df =  test_data.stn_df.copy()
vals = df[f'{target_id}_uar'].dropna().values
sdf = percentile_spread_convergence(vals)
years = list(set(df.index.year))

In [ ]:
from bokeh.models import ColorBar, LinearColorMapper, BasicTicker, LogColorMapper
from bokeh.transform import transform
from bokeh.palettes import RdBu11, RdBu10
from bokeh.models import ColumnDataSource, PrintfTickFormatter, CustomJSTickFormatter

deltas = [0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5, 1, 2, 5, 10]
P = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]
P = [10, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000]

# Convert axis values to strings for categorical plotting
deltas_str = [str(d) for d in deltas]
P_str = [str(p) for p in P]

# Prepare a matrix to store relative error
heatmap_data = np.zeros((len(deltas), len(P)))
sample_std = np.std(vals, ddof=0)
# Fill the matrix
for i, d in enumerate(deltas):
    for j, p in enumerate(P):
        lower = d
        upper = 100 - d
        percentiles = np.linspace(lower, upper, p)
        q_vals = np.percentile(vals, percentiles)
        spread_std = np.std(q_vals)
        rel_error = 100*(spread_std - sample_std) / sample_std
        heatmap_data[i, j] = rel_error

# Prepare dataframe for Bokeh
df = pd.DataFrame(heatmap_data, index=deltas_str, columns=P_str)
df.index.name = 'delta'
df.columns.name = 'P'
df = df.stack().rename("rel_error").reset_index()


In [ ]:
df['log_rel_error'] = np.log(df['rel_error'].abs())
source = ColumnDataSource(df)
# Set up color mapper
mapper = LinearColorMapper(palette=RdBu11,
                           low=df.log_rel_error.min(),
                           high=df.log_rel_error.max())

# Create figure
p = figure(x_range=P_str,
           y_range=list(reversed(deltas_str)),
           x_axis_label="Number of Percentiles",
           y_axis_label="Symmetric Exclusion from [0, 100] (%)",
           title="Relative Error of Quantile-based Std Dev vs Sample Std Dev",
           width=900,
           height=400,
           tools="")

# Add heatmap rectangles
p.rect(x="P", y="delta", width=1, height=1, source=source,
       line_color=None, fill_color=transform('log_rel_error', mapper))

formatter = CustomJSTickFormatter(code="""
    return Math.exp(tick).toFixed(1);
""")

# Add color bar
color_bar = ColorBar(color_mapper=mapper, ticker=BasicTicker(desired_num_ticks=11),
                     label_standoff=12, location=(0, 0), 
                     # formatter=PrintfTickFormatter(format="%d%%"),
                     formatter=formatter,
                     title='Relative (Absolute) Error in Quantile-based Stdev. [%]'
                    )
p.add_layout(color_bar, 'right')
hover = HoverTool(
    tooltips=[
        ("Percentiles", "@P"),
        ("Exclusion (%)", "@delta_float"),
        ("Rel. Error", "@rel_error{0.0}%")
    ],
    mode='mouse'
)

p.add_tools(hover)
show(p)

In [ ]:
def compare_quantile_std_and_mse_bias(stn, values):
    """
    Compute % relative error in standard deviation and MSE for two quantile sampling strategies:
    
    - Case A: np.arange(1, 99, 1)
    - Case B: [1, 5, 10, 50, 90, 95, 99]

    Parameters:
        stn (str): station ID
        values (array-like): 1D array of numeric values

    Returns:
        dict: {
            'stn_id': str,
            'a_err': float (% rel error in std dev),
            'b_err': float,
            'a_mse': float (MSE of q_vals_a vs true q_vals),
            'b_mse': float,
            'sample_std': float,
            'std_case_a': float,
            'std_case_b': float
        }
    """
    values = np.asarray(values)
    values = values[~np.isnan(values)]
    sample_std = np.std(values)

    percentiles_a = np.arange(1, 99, 1)
    # percentiles_b = [1, 5, 10, 50, 90, 95, 99]
    percentiles_b = [0.1,0.5,1,2,5,10,20, 40, 60, 80, 90, 95, 98, 99, 99.5, 99.9]

    # "Observed" quantiles
    q_vals_a = np.percentile(values, percentiles_a)
    q_vals_b = np.percentile(values, percentiles_b)

    std_a = np.std(q_vals_a)
    std_b = np.std(q_vals_b)

    rel_error_a = 100 * (std_a - sample_std) / sample_std
    rel_error_b = 100 * (std_b - sample_std) / sample_std

    return {
        'stn_id': stn,
        'a_err': rel_error_a,
        'b_err': rel_error_b,
        'sample_std': sample_std,
        'std_case_a': std_a,
        'std_case_b': std_b
    }

In [ ]:
all_results = []
stn_set = [s for s in context.official_ids if s in daymet_concurrent_stations]
for stn in stn_set:
    target_da= stn_da_dict[stn]
    test_data = StationData(context, stn)
    
    # get the unique UAR values
    df =  test_data.stn_df.copy()
    vals = df[f'{stn}_uar'].dropna().values
    log_vals = np.log(vals)
    # sdf = percentile_spread_convergence(vals)
    res = compare_quantile_std_and_mse_bias(stn, log_vals)
    all_results.append(res)
    if len(all_results) % 50 == 0:
        print(f'{len(all_results)}/{len(stn_set)}')
    

In [ ]:
def empirical_cdf(values):
    """Compute x and y for plotting empirical CDF."""
    values = np.sort(values)
    n = len(values)
    y = np.linspace(0, 1, n, endpoint=False)
    return values, y

In [ ]:
df_errs = pd.DataFrame(all_results)
df_errs.head()

In [ ]:
# Compute CDFs
x_a, y_a = empirical_cdf(df_errs['a_err'].dropna())
x_b, y_b = empirical_cdf(df_errs['b_err'].dropna())

# Create Bokeh plot
p = figure(
    width=600, height=400,
    x_axis_label="Relative Error (%)",
    y_axis_label="Empirical CDF",
    title="Empirical CDFs of Quantile-based MSE and Std Dev Errors"
)

line_a = p.line(x_a, y_a, line_width=2, color="green", line_dash='dashed', legend_label="(Booker & Snelder 2012, 2014)")
line_b = p.line(x_b, y_b, line_width=2, color="blue", legend_label="(Castellarin et al. 2007)")

# Optional: fine tune
p.legend.location = "bottom_right"
p.legend.click_policy = "hide"
p.legend.background_fill_alpha = 0.5
p.grid.grid_line_alpha = 0.3
p = dpf.format_fig_fonts(p, font_size=14)
show(p)


In [ ]:
x1, y1 = df_errs['sample_std'], df_errs['a_err']
x2, y2 = df_errs['sample_std'], df_errs['b_err']
# x_b, y_b = empirical_cdf(df_errs['b_err'].dropna())

# Create Bokeh plot
p = figure(
    width=600, height=400,
    x_axis_label="Sample Stdev.",
    y_axis_label="Relative Error (%)",
    title=""
)

p.scatter(x1, y1, line_width=2, color="green", alpha=0.5, size=1.5,
                   legend_label="(Booker & Snelder 2012, 2014)")
p.scatter(x2, y2, line_width=2, color="blue",  alpha=0.5, size=1.5,
                   legend_label="(Castellarin et al. 2007)")

# Optional: fine tune
p.legend.location = "bottom_right"
p.legend.click_policy = "hide"
p.legend.background_fill_alpha = 0.5
p.grid.grid_line_alpha = 0.3
p = dpf.format_fig_fonts(p, font_size=14)
show(p)